<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# AWS Bedrock
In this notebook we are going to inspect how to use AWS Bedrock, amazon's fully managed generative AI service to perform Text Generation!

## API connection

Before starting, you need to initiate and configure **boto3**. 

We start by importing the boto3 library, which is the Amazon Web Services (AWS) SDK for Python. This SDK allows you to write software that makes use of AWS services

Subsequently we create a client for the Bedrock service. This client is configured with the necessary credentials (AWS access key ID and AWS secret access key) and region (us-east-1) we want to operate in. Note that most bedrock services currently only work in the **us-east-1** region

To verify that everything worked, we call *list_foundation_models()* to list all available models. 


In [1]:
import boto3
bedrock = boto3.client(aws_access_key_id="",
                       aws_secret_access_key="",
                       region_name="us-east-1", service_name='bedrock')
bedrock.list_foundation_models()


{'ResponseMetadata': {'RequestId': 'd8a5207a-23bb-4638-97d6-cb5b2e0f7253',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 16 Dec 2023 09:16:11 GMT',
   'content-type': 'application/json',
   'content-length': '17836',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'd8a5207a-23bb-4638-97d6-cb5b2e0f7253'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

## Model Access
As of today, you need to [request](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access.html) models prior to using them.
To do so, navigate to https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/modelaccess and request access to the models of your choice.

[Here](https://us-east-1.console.aws.amazon.com/bedrock/home?region=us-east-1#/providers) you can find an overview over the available models.

In this notebook we will use the amazon.titan-text-express-v1 model.

In [2]:
bedrock.get_foundation_model(modelIdentifier='amazon.titan-text-express-v1')


{'ResponseMetadata': {'RequestId': 'b7ec8de0-9a74-44d2-af77-f3c0a816bd8e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 16 Dec 2023 09:16:28 GMT',
   'content-type': 'application/json',
   'content-length': '402',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'b7ec8de0-9a74-44d2-af77-f3c0a816bd8e'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-text-express-v1',
  'modelId': 'amazon.titan-text-express-v1',
  'modelName': 'Titan Text G1 - Express',
  'providerName': 'Amazon',
  'inputModalities': ['TEXT'],
  'outputModalities': ['TEXT'],
  'responseStreamingSupported': True,
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'ACTIVE'}}}

## Text Completion

The first task is to perform text completion. Text completion is an AI-driven process where the system automatically completes sentences or paragraphs based on a given text prompt.

### How Text Completion Works in AWS Bedrock

1. **Provide a Prompt**: Start by providing an initial text prompt. This could be a sentence, a question, or even a few words. The prompt should be relevant to the context of the completion you need.

2. **Configure the API**: Use the AWS Bedrock API to send the prompt. You may need to configure parameters such as the length of completion, the style of writing, or any specific instructions you want the AI to follow.

3. **Receive the Completion**: The AWS Bedrock service processes the prompt and returns a text completion. This completion is generated in real-time and aims to be a natural extension of the input prompt.

4. **Refinement and Iteration**: Depending on your needs, you might refine the initial prompt or adjust the parameters for different results. Iteration can help tailor the output to better suit your requirements.

### Examples of Text Completion

Here are a few examples of text completion tasks that can be accomplished with AWS Bedrock:

- **Creative Writing**: Given the start of a story, AWS Bedrock can continue the narrative, maintaining the tone and style of the original text.

- **Email Drafting**: For business communications, provide a brief outline or key points, and AWS Bedrock can formulate a complete, professional email.

- **Code Suggestions**: When programming, input a part of the code, and the service can suggest logical code completions or bug fixes.

- **Language Translation**: Begin with a sentence in one language, and AWS Bedrock can complete the translation, ensuring linguistic and contextual accuracy.

### Best Practices

To get the most out of text completion with AWS Bedrock:

- **Provide Clear and Contextual Prompts**: The quality of the output is largely dependent on the input. Clear and contextual prompts yield better results.

- **Iterative Approach**: Experiment with different prompts and settings. Iterative refinement can significantly enhance the outcome.

- **Understand Limitations**: While powerful, generative AI models have limitations. Be aware of these and review outputs critically, especially for sensitive or critical applications.

Let's get started by asking an absolute trivial question!

In [3]:
question = "What is the purpose of life?"

## Titan API Call
To invoke the model, we need to access the **bedrock-runtime** [service](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html)


In [4]:
bedrock_runtime = boto3.client(aws_access_key_id="",
                               aws_secret_access_key="",
                               region_name="us-east-1",
                               service_name='bedrock-runtime')


It provides the [invoke_model](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/invoke_model.html) function to run inference.

To use this function, we need to provide at least the following arguments:

- body
- modelId

The request body looks diffrerent depending on the model you use. The parameters are listed [here](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters.html).

### Titan Express Parameters
AWS' titan text models accept the following parameters:

- `inputText` [required]: string
  - Input text (prompt)
- `textGenerationConfig`[optional]:
  - Configuration settings for text generation.
  - `temperature`: float
    - This controls the randomness in the text generation process. A higher temperature results in more random outputs, while a lower temperature produces more predictable text.
  - `topP`: float
    - This parameter, also known as "nucleus sampling," controls the diversity of the generated text. It sets a threshold to include the most likely next words, cumulatively adding up to the specified probability 'P'. A lower value ignores less probable options.
  - `maxTokenCount`: int
    - This specifies the maximum number of tokens that the generated text can contain.
  - `stopSequences`: [string]
    - An array of string sequences that, when detected, will prompt the text generation to stop. This is useful for defining specific endpoints in generated text. "Use the | (pipe) character to separate different sequences (maximum 20 characters)."
   
Thus, the Titan model expects the following jsonified request body:
```
{
    "inputText": string,
    "textGenerationConfig": {
        "temperature": float,  
        "topP": float,
        "maxTokenCount": int,
        "stopSequences": [string]
    }
}
```

Note you can always use the default values for the textGenerationConfig


| Category              | Parameter        | JSON field format | Minimum | Maximum | Default |
|-----------------------|------------------|-------------------|---------|---------|---------|
| Randomness and diversity | Temperature     | `temperature`     | 0       | 1       | 0       |
|                       | Top P            | `topP`            | 0       | 1       | 1       |
| Length                | Response length  | `maxTokenCount`   | 0       | 8000    | 512     |



Alright! After all this information, let's send the first request to Titan using the default values! 

In [5]:
import json

body = json.dumps({
    "inputText": question,
})
print(body)

{"inputText": "What is the purpose of life?"}


In [6]:
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-express-v1")

In [7]:
response

{'ResponseMetadata': {'RequestId': '330a21a6-cdbc-4daa-9ce2-94f084883c88',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 16 Dec 2023 09:27:00 GMT',
   'content-type': 'application/json',
   'content-length': '523',
   'connection': 'keep-alive',
   'x-amzn-requestid': '330a21a6-cdbc-4daa-9ce2-94f084883c88',
   'x-amzn-bedrock-invocation-latency': '3141',
   'x-amzn-bedrock-output-token-count': '79',
   'x-amzn-bedrock-input-token-count': '7'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x10f101c90>}

### Model response
The result is stored within the response body, that you can access as shown below:

In [8]:
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 7,
 'results': [{'tokenCount': 79,
   'outputText': '\nThe purpose of life varies from person to person. Some people find purpose in pursuing their passions, while others find purpose in helping others or making a positive impact on the world. Some people may find purpose in religious or spiritual beliefs, while others may find purpose in personal growth and self-discovery. Ultimately, the purpose of life is a personal journey that requires self-reflection and exploration.',
   'completionReason': 'FINISH'}]}

**Important: Note that json.loads(response.get('body').read()) can only be executed once. After that the buffer is empty. So make sure to store the result!**


- `inputTextTokenCount`: int
  -  Number of tokens in the input text.
- `results`: array of objects
    - `tokenCount`: int
      - Number of tokens in the generated output. 
    - `outputText`: string
      - This string contains the actual text that was generated or outputted. The text includes newline characters (`\n`) to denote new lines or paragraphs in the generated text.
    - `completionReason`: string
      - Reason why the response terminated. Either:
      - `FINISHED` – The response was fully generated.
      - `LENGTH` – The response was truncated because of maxTokenCount.



In [11]:
print(response_body["results"][0]["outputText"])


The purpose of life varies from person to person. Some people find purpose in pursuing their passions, while others find purpose in helping others or making a positive impact on the world. Some people may find purpose in religious or spiritual beliefs, while others may find purpose in personal growth and self-discovery. Ultimately, the purpose of life is a personal journey that requires self-reflection and exploration.


**What a beautiful answer!**
However, a shorter answer would be nice, so let's reduce the max tokens

In [12]:
body = json.dumps({
    "inputText": question,
    "textGenerationConfig": {
        "maxTokenCount": 20
    }
})
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-express-v1")

In [13]:
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 7,
 'results': [{'tokenCount': 20,
   'outputText': '\nThe purpose of life varies from person to person. Some people find purpose in pursuing their passions,',
   'completionReason': 'LENGTH'}]}

Note how the completionReason changed from FINISHED to LENGTH indicating that the answer was truncated because of maxTokenCount

Let's try to get a better understanding of the parameters `temperature`and `topP` using the following prompt:

In [39]:
prompt = "Tell me a story"

First, a basic approach with a very low temperature and topP to decrease randomness and ignore less probable results

In [44]:
body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0,
        "topP": 0.01,  # needs to be larger than 0
        "maxTokenCount": 512
    }
})
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 4,
 'results': [{'tokenCount': 512,
   'outputText': '\n"Once upon a time, there was a young man named John who lived in a small village on the outskirts of a great city. John was a simple man, with a heart full of kindness and a mind full of dreams. He had always wanted to travel the world, to see all the wonders that it had to offer, but he had never had the means to do so.\n\nOne day, while John was out for a walk in the countryside, he stumbled upon a hidden cave. Inside the cave, he found a beautiful crystal that glowed with an otherworldly light. John was amazed by the crystal\'s beauty and he picked it up, feeling a strange energy coursing through his body.\n\nAs John left the cave and returned to his village, he felt a newfound sense of confidence and strength. He knew that the crystal had given him a special gift, and he was determined to use it to make his dreams a reality.\n\nOver the next few weeks, John worked tirelessly to save up enough money to t

In [45]:
print(response_body["results"][0]["outputText"])


"Once upon a time, there was a young man named John who lived in a small village on the outskirts of a great city. John was a simple man, with a heart full of kindness and a mind full of dreams. He had always wanted to travel the world, to see all the wonders that it had to offer, but he had never had the means to do so.

One day, while John was out for a walk in the countryside, he stumbled upon a hidden cave. Inside the cave, he found a beautiful crystal that glowed with an otherworldly light. John was amazed by the crystal's beauty and he picked it up, feeling a strange energy coursing through his body.

As John left the cave and returned to his village, he felt a newfound sense of confidence and strength. He knew that the crystal had given him a special gift, and he was determined to use it to make his dreams a reality.

Over the next few weeks, John worked tirelessly to save up enough money to travel the world. He sold his possessions, worked odd jobs, and even begged for change 

Let's compare that to a more creative model by  increasing temperature and topP!

In [48]:
body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 1,
        "topP": 1,
        "maxTokenCount": 512
    }
})
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 4,
 'results': [{'tokenCount': 279,
   'outputText': '\n\'When Johnny Comes Marching Home\' is a popular marching tune written by John Philip Sousa in 1917. The song is not explicitly about the Vietnam War, but it has been commonly associated with it due to its patriotic and militaristic themes.\n\nThe song is set in the key of C major and has a tempo of 68 beats per minute. It features a marching rhythm with a trumpet solo and a catchy melody. The lyrics are as follows:\n\nWhen Johnny comes marching home again\nHum drum drum, boys, hum drum drum\nWhen Johnny comes marching home again\nWe’ll give him a hoo-rah, hoo-rah, hoo-rah\nWe’ll drive the girls wild, we’ll drive the boys wild\nWhen Johnny comes marching home again\n\nThe song has been performed by numerous marching bands, including the United States Marine Band and the 76th Infantry Division Band. It has also been featured in a number of films, including the 1984 movie "Platoon."\n\nDespite its association

In [49]:
print(response_body["results"][0]["outputText"])


'When Johnny Comes Marching Home' is a popular marching tune written by John Philip Sousa in 1917. The song is not explicitly about the Vietnam War, but it has been commonly associated with it due to its patriotic and militaristic themes.

The song is set in the key of C major and has a tempo of 68 beats per minute. It features a marching rhythm with a trumpet solo and a catchy melody. The lyrics are as follows:

When Johnny comes marching home again
Hum drum drum, boys, hum drum drum
When Johnny comes marching home again
We’ll give him a hoo-rah, hoo-rah, hoo-rah
We’ll drive the girls wild, we’ll drive the boys wild
When Johnny comes marching home again

The song has been performed by numerous marching bands, including the United States Marine Band and the 76th Infantry Division Band. It has also been featured in a number of films, including the 1984 movie "Platoon."

Despite its association with the Vietnam War, "When Johnny Comes Marching Home" remains a popular and beloved patriot

Well... You can see that by increasing temperature and topP your stories are getting more "random". 
Let's try something in between:

In [59]:
body = json.dumps({
    "inputText": prompt,
    "textGenerationConfig": {
        "temperature": 0.8,
        "topP": 0.3,
        "maxTokenCount": 512
    }
})
response = bedrock_runtime.invoke_model(body=body, modelId="amazon.titan-text-express-v1")
response_body = json.loads(response.get('body').read())
response_body

{'inputTextTokenCount': 4,
 'results': [{'tokenCount': 342,
   'outputText': '\n"Once upon a time, there was a little girl named Lily. She loved to dance and spent most of her days twirling and leaping around the house. One day, her parents decided to enroll her in a dance class. Lily was thrilled and couldn\'t wait to start.\n\nAt first, Lily was nervous. She didn\'t know the other students and felt shy. But her teacher was patient and kind, and soon Lily began to feel more comfortable. She learned new dance moves and practiced every day.\n\nAs the weeks went by, Lily\'s confidence grew. She started to perform in front of her classmates and even in small local competitions. She loved the feeling of being on stage, of feeling the music, and of sharing her passion with others.\n\nOne day, a famous dance company came to town to perform. Lily was thrilled when her teacher told her that she had been invited to perform with them. She practiced for weeks, perfecting her moves and building he

In [60]:
print(response_body["results"][0]["outputText"])


"Once upon a time, there was a little girl named Lily. She loved to dance and spent most of her days twirling and leaping around the house. One day, her parents decided to enroll her in a dance class. Lily was thrilled and couldn't wait to start.

At first, Lily was nervous. She didn't know the other students and felt shy. But her teacher was patient and kind, and soon Lily began to feel more comfortable. She learned new dance moves and practiced every day.

As the weeks went by, Lily's confidence grew. She started to perform in front of her classmates and even in small local competitions. She loved the feeling of being on stage, of feeling the music, and of sharing her passion with others.

One day, a famous dance company came to town to perform. Lily was thrilled when her teacher told her that she had been invited to perform with them. She practiced for weeks, perfecting her moves and building her confidence.

The day of the performance arrived, and Lily stepped onto the stage. She 